# Annotations made by the model

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt 
from project_paths import paths 
from ipywidgets import interact, IntSlider

from lane_detection_medium.inference import DetectionInference
from lane_detection_medium.utils.fs import write_image, get_date_string, write_yolo_labels
from lane_detection_medium.utils.viz import render_bbox
from lane_detection_medium.utils.video_processing import VideoReader, VideoWriter
from lane_detection_medium.types.box_types import YoloBbox

%matplotlib inline

## Trained Model Loading

In [ ]:
EXP_NAME = "train-2023-07-05"

CHECKPOINT_DPATH = paths.yolo_dpath / "LaneMarkingsDetection" / EXP_NAME / "weights"
MODEL_FPATH = CHECKPOINT_DPATH / "best.pt"

inference = DetectionInference.from_file(
    str(MODEL_FPATH), 
    device="cuda:0", 
    img_size=(640, 640)
)

## Video Processing

In [ ]:
bg_map = [(0, 249), (3839, 4080), (4122, 4240), (4588, 10319)]
#         (0, 267), (3839, 4090), (4119, 4243), (4581, 10325)

video_fname = "astrahan1.mp4"
video_fpath = paths.data / "videos" / video_fname

out_dpath = paths.yolo_dpath / "data" / get_date_string() / "train"

out_img_dpath = out_dpath / "images"
out_img_dpath.mkdir(parents=True, exist_ok=True)

out_lbl_dpath = out_dpath / "labels"
out_lbl_dpath.mkdir(parents=True, exist_ok=True)

with VideoReader(video_fpath, verbose=True) as reader:
    for frame_idx, frame_img in enumerate(reader.get_frames()):
        if frame_idx % 2 != 0:
            continue 

        frame_fstem = f"{video_fpath.stem}_frame_{frame_idx:06d}"
        write_image(out_img_dpath / f"{frame_fstem}.PNG", frame_img)

        is_background = False 
        for f_range in bg_map:
            if f_range[0] <= frame_idx <= f_range[1]:
                is_background = True 
                break 

        if is_background:
            continue

        detections = inference.detect([frame_img], conf=0.25)[0]

        if not len(detections):
            continue

        yolo_labels = []
        for det in detections:
            yolo_bbox = YoloBbox.from_bbox(det.bbox, *frame_img.shape[:2])
            yolo_label = [det.label_id] + yolo_bbox.xywh.tolist()
            yolo_labels.append(yolo_label)

        write_yolo_labels(out_lbl_dpath / f"{frame_fstem}.txt", yolo_labels)

In [ ]:
img_dpath = paths.yolo_dpath / "data" / "2023_07_10" / "train" / "images"
lbl_dpath = paths.yolo_dpath / "data" / "2023_07_10" / "train" / "labels"

imgs = list(img_dpath.glob("*.PNG"))
lbls = list(lbl_dpath.glob("*.txt"))

len(imgs), len(lbls)